In [41]:
import string 
import json 

import pandas as pd
import nltk
import spacy

import datetime 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nlp = spacy.load('de_core_news_sm') 

#Definition für top 10 keywords 
def get_top_n_kw(document, vectorizer, n):
    matrix = vectorizer.transform([document])
    scores = pd.DataFrame(matrix[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
    scores = scores[scores["TF-IDF"]!=0]
    return {k: v["TF-IDF"] for k,v in scores.sort_values("TF-IDF", ascending=False)[:n].to_dict("index").items()}
    


c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.conda\envs\jemen-scraper\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\ddoro\.

In [2]:
#Preprocessing mit Lemmatisierung, Umlauten und stemming
with open ("../analysis/custom-replacements.json", "r") as file:
    replacements = json.load(file)

def preprocess(article):
    document = " ".join([article["text"], article["title"], article["surtitle"]]).lower()
    document.replace("sammads", "sammad")
    for sw in ["\n"]:
        document = document.replace(sw, " ")
    # part of speech tagging und lemmatization
    document = " ".join([t.lemma_ for t in nlp(document) if t.pos_ in ["NOUN", "PROPN", "VERB", "X"]]).lower()
    for k,v in replacements.items():
        document = document.replace(k, v)
    for n in range(10):
        document = document.replace(str(n), "")
    article["preprocessed_text"] = document 
    return article

In [3]:
# preprocess corpus and save preprocessed text in article (corpus is a list of article dictionaries)
with open("../data/sz/all-relevant-fulltext.json", "r", encoding="utf-8") as file:
    s = file.read()
    articles_sz = json.loads(s)

with open("../data/faz/all-relevant-fulltext.json", "r", encoding="utf-8") as file:
    s = file.read()
    articles_faz = json.loads(s)

corpus = [preprocess(article) for article in articles_faz + articles_sz]


In [4]:
# generate tfidf matrices using scikit learn vectorizer and generate n keywords for every article and save it in article dict
nltk.download('stopwords')
german_stop_words = stopwords.words('german')

tfidf_vect = TfidfVectorizer(stop_words=german_stop_words)
tfidf_vect = tfidf_vect.fit([article["preprocessed_text"] for article in corpus])

for article in corpus:
    article["keywords"] = get_top_n_kw(article["preprocessed_text"], tfidf_vect, 10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddoro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# generate a list of all keywords occuring onces
final_keywords = []

for article in corpus:
    final_keywords += list(article["keywords"].keys())


print(f"A total of {len(final_keywords)} keywords were produced" )
unique_keywords = list(set(final_keywords))
print(f"An {len(unique_keywords)} unique ones")


A total of 6830 keywords were produced
An 3364 unique ones


In [6]:
# Liste von keywords die öfter als einmal vorkommen
from collections import Counter
c = Counter(final_keywords)
all_kw_with_frequency =[(x,v) for x,v in dict(c).items()]
keywords_that_occur_min_twice = {x:v for x,v in dict(c).items() if v>1}
singular_occurences = [x for x,v in dict(c).items() if v==1]

In [7]:
with open("tmp.txt", "w", encoding="utf-8") as f:
    json.dump(unique_keywords, f, indent=2, ensure_ascii=False)

In [9]:
# create dataframe w/ random sample of articles and respective keywords
import random
sample = [{"link":a["link"], "kws": ", ".join(a["keywords"].keys())} for a in random.sample(corpus, 30)]
pd.DataFrame(sample).to_csv("random_articles_for_categorie_gen.tsv", sep="\t")



In [17]:
# create dataframe w/ random sample of 10 articles and respective keywords per year
corpus_df = pd.DataFrame(corpus)
years = list(set([(d.year) for d in corpus_df["date"]]))

samples_by_year = pd.DataFrame()

for year in years:
    tmp = pd.DataFrame([article for article in corpus if year == article["date"].year])
    print(len(samples_by_year))
    if not len(samples_by_year):
        samples_by_year = tmp.sample(10)
    else:
        samples_by_year = pd.concat([samples_by_year, tmp.sample(10)])

samples_by_year.to_csv("samples_by_year.tsv", sep="\t", index=False)

0
10
20
30
40
50
60
70


In [ ]:
with open ("corpus.json", "w") as file:
    json.dump(corpus, file, indent = 2)

